## Human or Bot Detection Using Bidirectional LSTM


In [51]:
import pandas as pd

In [52]:
df=pd.read_csv('preprocessed_filename.csv')

In [53]:
df.head()

,Texts,Texts2,corpus,onehot_repr
0,Fantastic picture quality and great value for ...,human,fantast pictur qualiti great valu money,"[2740, 673, 1124, 4332, 2133, 420]"
1,Good,human,good,[2183]
2,Superb quality ❤️,human,superb qualiti,"[4659, 1124]"
3,Super,human,super,[2388]
4,Just ok (only),human,ok,[2006]


In [54]:
###Drop Nan Values
df=df.dropna()


In [55]:
## Get the Independent Features

X=df.drop(['Texts2','Texts','corpus'],axis=1)

In [56]:
## Get the Dependent features
y=df['Texts2']

In [57]:
y.value_counts()

human    272
bot      249
Name: Texts2, dtype: int64

In [58]:
X.shape

(521, 1)

In [59]:
y.shape

(521,)

In [60]:
import tensorflow as tf

In [61]:
tf.__version__

'2.12.0'

In [62]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [63]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [64]:
import nltk
import re


In [65]:
df['corpus']

0                fantast pictur qualiti great valu money
1                                                   good
2                                         superb qualiti
3                                                  super
4                                                     ok
                             ...                        
518    advanc ai grow concern impact employ expert pr...
519    ai languag model person perspect belief provid...
520    doubt ai signific impact employ futur job repe...
521    moreov ai improv product effici lead econom gr...
522    overal ai may lead job displac industri also c...
Name: corpus, Length: 521, dtype: object

In [66]:
onehot_repr=[one_hot(words,voc_size)for words in df['corpus']] 
onehot_repr

[[3656, 2891, 2293, 1540, 1293, 1526],
 [1233],
 [254, 2293],
 [1731],
 [2572],
 [1426, 944, 2081, 4885, 902, 1899, 4745, 1526, 4909, 1963, 2520, 4885],
 [4249, 1426, 3960, 2721, 1110, 1526, 2016],
 [4847, 4879, 1888],
 [2375],
 [3833, 2081, 2749, 1307],
 [3656, 2891, 2293, 1540, 1293, 1526],
 [1233],
 [254, 2293],
 [1731],
 [2572],
 [1426, 944, 2081, 4885, 902, 1899, 4745, 1526, 4909, 1963, 2520, 4885],
 [4249, 1426, 3960, 2721, 1110, 1526, 2016],
 [4847, 4879, 1888],
 [2375],
 [3833, 2081, 2749, 1307],
 [1561, 1641],
 [3941,
  203,
  175,
  1006,
  203,
  1165,
  4705,
  4167,
  175,
  2233,
  3185,
  2155,
  2899,
  3199,
  1441,
  660,
  2191,
  4997,
  3916],
 [4251, 1452, 2891, 1949, 747, 1441, 660],
 [4251, 1452],
 [4251],
 [1452],
 [2891, 1949, 747, 1441, 660],
 [4465],
 [3656, 2891, 2293, 1540, 1293, 1526],
 [3491, 1380, 3503, 2107, 3041],
 [3491, 1380, 3503],
 [2107, 3041],
 [164, 1193],
 [1233],
 [3491, 1380, 2699, 2107, 3041],
 [3491, 1380, 2699],
 [2107, 3041],
 [1193],
 [

In [67]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1540 1293 1526]
 [   0    0    0 ...    0    0 1233]
 [   0    0    0 ...    0  254 2293]
 ...
 [3490 4200 4718 ... 4006 2342 4343]
 [4416 4061 3214 ... 2743 1110 3263]
 [4304 2227  834 ... 1839 4861 4329]]


In [68]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
len(embedded_docs),y.shape

(521, (521,))

In [70]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
le = LabelEncoder()

# Fit the LabelEncoder to the categorical data and transform it into numerical labels
y = le.fit_transform(y)

# Print the numerical labels
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [71]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [72]:
X_final.shape,y_final.shape

((521, 20), (521,))

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [74]:
X_train

array([[   0,    0,    0, ...,  375, 2008,  415],
       [   0,    0,    0, ...,    0,    0, 1731],
       [   0,    0,    0, ..., 3597, 3490, 2051],
       ...,
       [   0,    0,    0, ..., 2547, 1540, 4206],
       [   0,    0,    0, ..., 4582, 2675, 2051],
       [   0,    0,    0, ..., 1233, 2271, 1233]])

In [75]:
y_train

array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,

### Model Training

In [76]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
6/6 [==============================] - 4s 194ms/step - loss: 0.6872 - accuracy: 0.6991 - val_loss: 0.6819 - val_accuracy: 0.6860
Epoch 2/100
6/6 [==============================] - 0s 62ms/step - loss: 0.6641 - accuracy: 0.7650 - val_loss: 0.6643 - val_accuracy: 0.7093
Epoch 3/100
6/6 [==============================] - 0s 64ms/step - loss: 0.6036 - accuracy: 0.7908 - val_loss: 0.6215 - val_accuracy: 0.7267
Epoch 4/100
6/6 [==============================] - 0s 66ms/step - loss: 0.4709 - accuracy: 0.8138 - val_loss: 0.5577 - val_accuracy: 0.7267
Epoch 5/100
6/6 [==============================] - 0s 64ms/step - loss: 0.3348 - accuracy: 0.8739 - val_loss: 0.4218 - val_accuracy: 0.7791
Epoch 6/100
6/6 [==============================] - 0s 65ms/step - loss: 0.2232 - accuracy: 0.9054 - val_loss: 0.5157 - val_accuracy: 0.8372
Epoch 7/100
6/6 [==============================] - 0s 64ms/step - loss: 0.1539 - accuracy: 0.9484 - val_loss: 0.6458 - val_accuracy: 0.8256
Epoch 8/100
6/6 [==

6/6 [==============================] - 0s 63ms/step - loss: 1.3759e-04 - accuracy: 1.0000 - val_loss: 0.4437 - val_accuracy: 0.9070
Epoch 59/100
6/6 [==============================] - 0s 63ms/step - loss: 1.3017e-04 - accuracy: 1.0000 - val_loss: 0.4441 - val_accuracy: 0.9070
Epoch 60/100
6/6 [==============================] - 0s 64ms/step - loss: 1.4424e-04 - accuracy: 1.0000 - val_loss: 0.4461 - val_accuracy: 0.9070
Epoch 61/100
6/6 [==============================] - 0s 62ms/step - loss: 1.1805e-04 - accuracy: 1.0000 - val_loss: 0.4467 - val_accuracy: 0.9070
Epoch 62/100
6/6 [==============================] - 0s 62ms/step - loss: 1.4278e-04 - accuracy: 1.0000 - val_loss: 0.4461 - val_accuracy: 0.9070
Epoch 63/100
6/6 [==============================] - 0s 64ms/step - loss: 1.2861e-04 - accuracy: 1.0000 - val_loss: 0.4461 - val_accuracy: 0.9070
Epoch 64/100
6/6 [==============================] - 0s 62ms/step - loss: 1.0798e-04 - accuracy: 1.0000 - val_loss: 0.4457 - val_accuracy: 0.907

### Performance Metrics And Accuracy

In [77]:

y_pred1=model1.predict(X_test)

6/6 [==============================] - 1s 7ms/step


In [78]:
y_pred1

array([[9.9758685e-01],
       [9.9986607e-01],
       [9.9983817e-01],
       [9.9999714e-01],
       [9.9998033e-01],
       [3.5546083e-05],
       [9.9999779e-01],
       [6.2259140e-05],
       [1.7103839e-04],
       [2.3655305e-06],
       [9.9999475e-01],
       [9.9999529e-01],
       [5.4648604e-06],
       [9.5844096e-01],
       [9.9983853e-01],
       [6.3017433e-05],
       [1.9314057e-05],
       [9.9988222e-01],
       [9.9998808e-01],
       [9.9999565e-01],
       [9.9999672e-01],
       [9.9999458e-01],
       [9.9999565e-01],
       [9.9998975e-01],
       [9.9999511e-01],
       [9.9972486e-01],
       [9.9975860e-01],
       [6.0565577e-05],
       [8.8748893e-05],
       [9.9999404e-01],
       [2.7006516e-05],
       [9.9999672e-01],
       [2.9613178e-03],
       [9.9986243e-01],
       [6.3316525e-06],
       [9.9983007e-01],
       [9.9999660e-01],
       [9.9999249e-01],
       [2.4619940e-06],
       [9.9999648e-01],
       [9.9999744e-01],
       [9.999976

In [93]:
def process(x):
    if(x>=0.5):
        return "human"
    return "bot"

In [80]:
import numpy as np
y_pred1 = np.vectorize(process)(y_pred1)

In [81]:
len(X_test)

172

In [82]:
y_pred1

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
    

In [83]:
from sklearn.metrics import confusion_matrix

In [84]:
confusion_matrix(y_test,y_pred1)

array([[67, 11],
       [ 4, 90]], dtype=int64)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9127906976744186

In [86]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90        78
           1       0.89      0.96      0.92        94

    accuracy                           0.91       172
   macro avg       0.92      0.91      0.91       172
weighted avg       0.91      0.91      0.91       172



In [90]:
import pickle

# save the model to a file using pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model1, f)


In [91]:

# load the model from the file
with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

# make predictions on new data

y_pred = clf.predict(X_test)


6/6 [==============================] - 1s 8ms/step


In [92]:
y_pred

array([[9.9758685e-01],
       [9.9986607e-01],
       [9.9983817e-01],
       [9.9999714e-01],
       [9.9998033e-01],
       [3.5546083e-05],
       [9.9999779e-01],
       [6.2259140e-05],
       [1.7103839e-04],
       [2.3655305e-06],
       [9.9999475e-01],
       [9.9999529e-01],
       [5.4648604e-06],
       [9.5844096e-01],
       [9.9983853e-01],
       [6.3017433e-05],
       [1.9314057e-05],
       [9.9988222e-01],
       [9.9998808e-01],
       [9.9999565e-01],
       [9.9999672e-01],
       [9.9999458e-01],
       [9.9999565e-01],
       [9.9998975e-01],
       [9.9999511e-01],
       [9.9972486e-01],
       [9.9975860e-01],
       [6.0565577e-05],
       [8.8748893e-05],
       [9.9999404e-01],
       [2.7006516e-05],
       [9.9999672e-01],
       [2.9613178e-03],
       [9.9986243e-01],
       [6.3316525e-06],
       [9.9983007e-01],
       [9.9999660e-01],
       [9.9999249e-01],
       [2.4619940e-06],
       [9.9999648e-01],
       [9.9999744e-01],
       [9.999976

In [94]:
import numpy as np
y_pred1 = np.vectorize(process)(y_pred)

In [95]:
y_pred1[0:5]

array([['human'],
       ['human'],
       ['human'],
       ['human'],
       ['human']], dtype='<U5')